<a href="https://colab.research.google.com/github/ToshikiShimizu/PyTorch/blob/master/intro_dev/chapter4/4_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch
import torch
from torch import nn, optim
from torch.utils.data import(Dataset, DataLoader, TensorDataset)
import tqdm

    100% |████████████████████████████████| 519.5MB 22kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x58c34000 @  0x7fa5961802a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070


In [2]:
!wget https://github.com/lucidfrontier45/PyTorch-Book/raw/master/data/taco_and_burrito.tar.gz

--2018-11-11 04:31:04--  https://github.com/lucidfrontier45/PyTorch-Book/raw/master/data/taco_and_burrito.tar.gz
Resolving github.com (github.com)... 192.30.253.113, 192.30.253.112
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/lucidfrontier45/PyTorch-Book/master/data/taco_and_burrito.tar.gz [following]
--2018-11-11 04:31:05--  https://raw.githubusercontent.com/lucidfrontier45/PyTorch-Book/master/data/taco_and_burrito.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15292798 (15M) [application/octet-stream]
Saving to: ‘taco_and_burrito.tar.gz’

taco_and_burrito.ta 100%[===================>]  14.58M  28.8MB/s    in 0.5s    

2018-11-11 04:31

In [3]:
!tar -zxvf taco_and_burrito.tar.gz

taco_and_burrito/
taco_and_burrito/test/
taco_and_burrito/test/burrito/
taco_and_burrito/test/burrito/363.jpg
taco_and_burrito/test/burrito/372.jpg
taco_and_burrito/test/burrito/362.jpg
taco_and_burrito/test/burrito/386.jpg
taco_and_burrito/test/burrito/379.jpg
taco_and_burrito/test/burrito/367.jpg
taco_and_burrito/test/burrito/361.jpg
taco_and_burrito/test/burrito/384.jpg
taco_and_burrito/test/burrito/377.jpg
taco_and_burrito/test/burrito/365.jpg
taco_and_burrito/test/burrito/368.jpg
taco_and_burrito/test/burrito/374.jpg
taco_and_burrito/test/burrito/388.jpg
taco_and_burrito/test/burrito/378.jpg
taco_and_burrito/test/burrito/380.jpg
taco_and_burrito/test/burrito/370.jpg
taco_and_burrito/test/burrito/364.jpg
taco_and_burrito/test/burrito/385.jpg
taco_and_burrito/test/burrito/381.jpg
taco_and_burrito/test/burrito/383.jpg
taco_and_burrito/test/burrito/373.jpg
taco_and_burrito/test/burrito/366.jpg
taco_and_burrito/test/burrito/389.jpg
taco_and_burrito/test/burrito/376.jpg
taco_and_burrito

In [5]:
!pip install torchvision

    100% |████████████████████████████████| 61kB 2.7MB/s 
    100% |████████████████████████████████| 2.0MB 9.3MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
train_imgs = ImageFolder("taco_and_burrito/train/",transform=transforms.Compose([
    transforms.RandomCrop(224),
    transforms.ToTensor()]
))
test_imgs = ImageFolder("taco_and_burrito/test/",transform=transforms.Compose([
    transforms.RandomCrop(224),
    transforms.ToTensor()]
))
train_loader = DataLoader(
  train_imgs, batch_size = 32, shuffle=True)
test_loader = DataLoader(
  test_imgs, batch_size = 32, shuffle=True)

In [7]:
print(train_imgs.classes)

['burrito', 'taco']


In [8]:
print(train_imgs.class_to_idx)

{'burrito': 0, 'taco': 1}


In [10]:
from torchvision import models

net = models.resnet18(pretrained=True)
for p in net.parameters():
  p.requires_grad=False
fc_input_dim = net.fc.in_features
net.fc = nn.Linear(fc_input_dim, 2)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.torch/models/resnet18-5c106cde.pth
100%|██████████| 46827520/46827520 [00:06<00:00, 7611636.12it/s]


In [24]:
!pip install --no-cache-dir -I pillow

    100% |████████████████████████████████| 2.0MB 12.3MB/s 


In [0]:
def eval_net(net, data_loader, device="cpu"):
  net.eval()
  ys = []
  ypreds = []
  for x, y in data_loader:
    x = x.to(device)
    y = y.to(device)
    with torch.no_grad():
      _, y_pred = net(x).max(1)
    ys.append(y)
    ypreds.append(y_pred)
  ys = torch.cat(ys)
  ypreds = torch.cat(ypreds)
  acc = (ys == ypreds).float().sum() / len(ys)
  return acc.item()

In [0]:
def train_net(net, train_loader, test_loader, only_fc=True, optimizer_cls=optim.Adam,loss_fn=nn.CrossEntropyLoss(),n_iter=10,device="cpu"):
  train_losses = []
  train_acc = []
  val_acc = []
  if only_fc:
    optimizer = optimizer_cls(net.fc.parameters())
  else:
    optimizer = optimizer_cls(net.parameters())
  for epoch in range(n_iter):
    running_loss = 0.0
    net.train()
    n = 0
    n_acc = 0
    for i, (xx, yy) in tqdm.tqdm(enumerate(train_loader),total=len(train_loader)):
      xx = xx.to(device)
      yy = yy.to(device)
      h = net(xx)
      loss = loss_fn(h, yy)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      running_loss += loss.item()
      n += len(xx)
      _, y_pred = h.max(1)
      n_acc += (yy == pred).float().sum().item()
    train_losses.append(running_loss / i)
    val_acc.append(eval_net(net, test_loader, device))
    print (epoch, train_losses[-1], train_acc[-1],val_acc[-1],flush=True)
    
    
    

In [23]:

net.to("cuda:0")
train_net(net, train_loader, test_loader, n_iter=20,device="cuda:0")






  0%|          | 0/23 [00:00<?, ?it/s]

AttributeError: ignored